# LSTM 
Muhammad Farrel 1301184453 \
Haris Subekti 1301183323

# Upload Data File

In [13]:
from google.colab import files
uploaded = files.upload()

Saving model_lstm.h5 to model_lstm.h5


# Library

In [2]:
import re
import nltk
import numpy as np
from data import training_doc
nltk.download('punkt')

from nltk.tokenize import word_tokenize
from keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import to_categorical
from keras.preprocessing.sequence import pad_sequences

from keras.models import Sequential, load_model
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Embedding

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


# Preprocessing 

In [3]:
# data cleaning dan pemecahan kalimat
cleaned_data = re.sub(r'\W+', ' ', training_doc).lower()
tokens = word_tokenize(cleaned_data)

train_len = 4
text_sequences = []
for i in range(train_len,len(tokens)):
    seq = tokens[i-train_len:i]
    text_sequences.append(seq)

sequences = {}
count = 1
for i in range(len(tokens)):
    if tokens[i] not in sequences:
        sequences[tokens[i]] = count
        count += 1

In [7]:
# pemberian label pada tiap kata
tokenizer = Tokenizer()
tokenizer.fit_on_texts(text_sequences)
sequences = tokenizer.texts_to_sequences(text_sequences) 
sequences[:5]

[[7, 1, 186, 3],
 [1, 186, 3, 1],
 [186, 3, 1, 60],
 [3, 1, 60, 758],
 [1, 60, 758, 26]]

In [8]:
# collect informasi data untuk di proses  
vocabulary_size = len(tokenizer.word_counts)+1

n_sequences = np.empty([len(sequences),train_len], dtype='int32')
for i in range(len(sequences)):
    n_sequences[i] = sequences[i]

In [9]:
n_sequences.shape

(29552, 4)

In [10]:
# data split
train_x = n_sequences[:,:-1]
train_y = n_sequences[:,-1]

# mengubah data ke menjadi categorical
train_y = to_categorical(train_y, num_classes=vocabulary_size)
seq_len = train_x.shape[1]
train_x.shape

(29552, 3)

In [11]:
train_y[0]

array([0., 0., 0., ..., 0., 0., 0.], dtype=float32)

# Modelling

In [14]:
model = load_model("model_lstm.h5")

# model = Sequential()
# model.add(Embedding(vocabulary_size, seq_len, input_length=seq_len))
# model.add(LSTM(50,return_sequences=True))
# model.add(LSTM(50))
# model.add(Dense(50,activation='relu'))
# model.add(Dense(vocabulary_size, activation='softmax'))

# print(model.summary())

In [8]:
# training
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(train_x,train_y,epochs=300,verbose=1)
model.save("model_lstm.h5")

Epoch 1/300
924/924 [==============================] - 18s 15ms/step - loss: 4.3870 - accuracy: 0.2243
Epoch 2/300
924/924 [==============================] - 14s 15ms/step - loss: 4.2244 - accuracy: 0.2300
Epoch 3/300
924/924 [==============================] - 14s 15ms/step - loss: 4.1249 - accuracy: 0.2368
Epoch 4/300
924/924 [==============================] - 14s 15ms/step - loss: 4.0330 - accuracy: 0.2438
Epoch 5/300
924/924 [==============================] - 13s 15ms/step - loss: 3.9498 - accuracy: 0.2489
Epoch 6/300
924/924 [==============================] - 13s 14ms/step - loss: 3.8704 - accuracy: 0.2540
Epoch 7/300
924/924 [==============================] - 14s 15ms/step - loss: 3.7913 - accuracy: 0.2574
Epoch 8/300
924/924 [==============================] - 13s 14ms/step - loss: 3.7198 - accuracy: 0.2637
Epoch 9/300
924/924 [==============================] - 13s 15ms/step - loss: 3.6498 - accuracy: 0.2699
Epoch 10/300
924/924 [==============================] - 14s 15ms/step - l

In [46]:
model.evaluate(train_x,train_y)

924/924 [==============================] - 16s 16ms/step - loss: 1.1199 - accuracy: 0.7053


[1.1198678016662598, 0.7052652835845947]

# Testing model

In [18]:
test_data = input().strip().lower()

encoded = tokenizer.texts_to_sequences([test_data])[0]
pad_encoded = pad_sequences([encoded], maxlen=seq_len, truncating='pre')
print(encoded, pad_encoded)

for i in (model.predict(pad_encoded)[0]).argsort()[-3:][::-1]:
  pred_word = tokenizer.index_word[i]
  print("Next word suggestion:",pred_word)

what is the newest
[7, 8, 1, 3590] [[   8    1 3590]]
Next word suggestion: shock
Next word suggestion: term
Next word suggestion: format
